In [3]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import requests
import io
import os

# --- 數據載入 ---
# 由於 Dash Lab 假設數據集在本地，這裡加入自動下載邏輯確保程式碼可運行
DATA_URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

# 嘗試從網路下載數據集
try:
    response = requests.get(DATA_URL)
    response.raise_for_status() # 檢查 HTTP 錯誤
    spacex_df = pd.read_csv(io.BytesIO(response.content))
except requests.exceptions.RequestException as e:
    print(f"Error downloading data: {e}")
    print("Please ensure you have run the 'wget' command to download spacex_launch_dash.csv locally.")
    spacex_df = pd.DataFrame() # 創建空 DataFrame 以避免崩潰

if spacex_df.empty:
    print("DataFrame is empty. Cannot proceed with Dash app initialization.")
    exit()

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# 準備 Launch Site 下拉式選單的選項
launch_sites = spacex_df['Launch Site'].unique().tolist()
site_options = [{'label': 'All Sites', 'value': 'ALL'}]
site_options.extend([{'label': site, 'value': site} for site in launch_sites])

# --- Dash 應用程式初始化 ---
app = dash.Dash(__name__)

# --- 儀表板佈局 (TASK 1 & TASK 3) ---
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36',
                   'font-size': 40}),
    
    # TASK 1: Drop-down Input Component
    dcc.Dropdown(
        id='site-dropdown',
        options=site_options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

    # TASK 2: Pie chart output
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):", style={'padding': '10px 0'}),
    
    # TASK 3: Range Slider to Select Payload
    dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        marks={i: f'{i}' for i in range(0, 10001, 1000)},
        value=[min_payload, max_payload]
    ),

    # TASK 4: Scatter chart output
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# --- TASK 2: Pie Chart Callback 函數 ---
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # ALL Sites: 顯示所有地點的總成功發射數
        fig = px.pie(
            spacex_df, 
            values='class', 
            names='Launch Site', 
            title='Total Successful Launches By Site'
        )
        return fig
    else:
        # Specific Site: 顯示該地點成功 (1) vs. 失敗 (0) 的計數
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        
        # 使用 value_counts() 取得成功和失敗的數量
        success_counts = filtered_df['class'].value_counts().reset_index()
        success_counts.columns = ['class', 'count']
        
        fig = px.pie(
            success_counts, 
            values='count', 
            names='class', # 這裡 names 可以是 0 和 1，或者可以添加一個 'Outcome' 欄位來顯示文字
            title=f'Launch Outcomes for site {entered_site}'
        )
        # 為了更好的顯示，將 0 和 1 標籤轉換為 'Failure' 和 'Success'
        fig.update_traces(labels=['Failure', 'Success']) 
        return fig

# --- TASK 4: Scatter Chart Callback 函數 ---
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])
def update_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    
    # 根據 Payload Range 篩選數據
    filtered_payload_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    ]

    if entered_site == 'ALL':
        # ALL Sites: 繪製所有地點在選定酬載範圍內的散點圖
        title_text = f'Correlation between Payload and Success for All Sites (Payload: {low}kg - {high}kg)'
        fig = px.scatter(
            filtered_payload_df, 
            x='Payload Mass (kg)', 
            y='class', 
            color='Booster Version Category',
            title=title_text
        )
        return fig
    else:
        # Specific Site: 篩選特定站點的數據
        filtered_site_df = filtered_payload_df[filtered_payload_df['Launch Site'] == entered_site]
        
        # 繪製特定地點在選定酬載範圍內的散點圖
        title_text = f'Correlation between Payload and Success for {entered_site} (Payload: {low}kg - {high}kg)'
        fig = px.scatter(
            filtered_site_df, 
            x='Payload Mass (kg)', 
            y='class', 
            color='Booster Version Category',
            title=title_text
        )
        return fig
# --- 運行應用程式 ---
if __name__ == '__main__':
    # 建議在終端機中運行: python spacex_dash_app.py
    # 然後通過瀏覽器訪問 http://127.0.0.1:8050/
    print("Dash app starting on http://127.0.0.1:8050/")
    app.run(debug=True)

Dash app starting on http://127.0.0.1:8050/
